In [ ]:
pip install --upgrade xlrd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#장애인 콜 시스템 분석
import requests
import pprint
import json
import pandas as pd
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

#API KEY
my_key = '46454c487563686139374b4a4a486f'

start = '20220101'
end = '20220501'

dt_index = pd.date_range(start = start, end = end)
dt_list = dt_index.strftime("%Y%m%d").tolist()
Calltaxi1 = []

endpoint="http://openapi.seoul.go.kr:8088/" + my_key + "/xml/disabledCalltaxi/1/5/20220101"
resp = requests.get(endpoint)
xml_obj = bs4.BeautifulSoup(resp.text,'lxml-xml')
rows = xml_obj.findAll('item')
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값
# xml 안의 데이터 수집
for k in range(0, len(rows)):
      columns = rows[k].find_all()
      for j in range(0,len(columns)):
        if k ==0:
          name_list.append(columns[j].name)
        value_list.append(columns[j].text)
row_list.append(value_list)
Calltaxi1 = pd.DataFrame({'row_list' : row_list}, columns=name_list)

In [ ]:
for i in dt_list:
    endpoint=f"http://openapi.seoul.go.kr:8088/" + my_key + "/xml/disabledCalltaxi/1/5/" + i
    resp = requests.get(endpoint)
    xml_obj = bs4.BeautifulSoup(resp.text,'lxml-xml')
    rows = xml_obj.findAll('item')
    # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
    row_list = [] # 행값
    value_list = [] #데이터값
    # xml 안의 데이터 수집
    for k in range(1, len(rows)):
          columns = rows[k].find_all()
          for j in range(0,len(columns)):
            value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list = []
    l=0
    while True:
      lst=[]
      if l == len(row_list[0]): break
      lst.append(row_list[0][l:l+9])
      l+=9
      Calltaxi = pd.DataFrame(lst, columns=name_list)
      Calltaxi1 = pd.concat([Calltaxi1, Calltaxi])
      del Calltaxi

In [ ]:
import pandas as pd

pal = pd.read_excel("/content/drive/MyDrive/학술제/location.xlsx")

pal = pal[pal.시도 =='서울특별시']
pal = pal[['시군구','읍면동','위도','경도']]
for i in pal.index:
  if pal['읍면동'][i][-1] != "동":
    pal['읍면동'][i] = pal['읍면동'][i] + '동'
pal['지역'] = pal['시군구'] + pal['읍면동']
pal = pal.drop_duplicates(['지역'])

pal1 = pal[['지역','위도','경도']]
pal1 = pal1.rename(columns={'지역':'출발지역','위도':'출발위도', '경도':'출발경도'})

pal2 = pal[['지역','위도','경도']]
pal2 = pal2.rename(columns={'지역':'도착지역','위도':'도착위도', '경도':'도착경도'})

pal2

In [ ]:
Calltaxi2 = Calltaxi1.sort_values('receipttime')

In [ ]:
Calltaxi2 = Calltaxi2.drop_duplicates()

In [ ]:
Calltaxi3 = Calltaxi2[['no', 'startpos1', 'startpos2', 'endpos1', 'endpos2']]

In [ ]:
Calltaxi3 = Calltaxi3.rename(columns={'startpos1':'출발구', 'endpos1':'도착구', 'startpos2':'출발동', 'endpos2':'도착동'})

In [ ]:
Calltaxi3['출발지역'] = Calltaxi3['출발구'] + Calltaxi3['출발동']
Calltaxi3['도착지역'] = Calltaxi3['도착구'] + Calltaxi3['도착동']

In [ ]:
Calltaxi3 = Calltaxi3[['no', '출발지역', '도착지역']]

In [ ]:
data = pd.merge(Calltaxi3, pal1, on="출발지역")
data = pd.merge(data, pal2, on="도착지역")
data = data.drop_duplicates()
data

In [ ]:

start = pd.read_csv("/content/drive/MyDrive/학술제/startD.csv")


end = pd.read_csv("/content/drive/MyDrive/학술제/endD.csv")

In [ ]:
start = start.rename(columns={'Unnamed: 0':'출발지역', 'startpos2':'출발합계'})
end = end.rename(columns={'Unnamed: 0':'도착지역', 'endpos2':'도착합계'})

In [ ]:
pal3 = pd.read_excel("/content/drive/MyDrive/학술제/location.xlsx")

pal3 = pal3[pal3.시도 =='서울특별시']
pal3 = pal3[['시군구','읍면동']]
pal3 = pal3.rename(columns={'읍면동':'출발지역'})

pal4 = pd.read_excel("/content/drive/MyDrive/학술제/location.xlsx")

pal4 = pal4[pal4.시도 =='서울특별시']
pal4 = pal4[['시군구','읍면동']]
pal4 = pal4.rename(columns={'읍면동':'도착지역'})

sumofstart = pd.merge(start, pal3, on="출발지역")
sumofstart['출발지역'] = sumofstart['시군구'] + sumofstart['출발지역']
sumofstart = sumofstart[['출발지역' ,'출발합계']]

sumofstart

In [ ]:
sumofend = pd.merge(end, pal4, on="도착지역")
sumofend['도착지역'] = sumofend['시군구'] + sumofend['도착지역']
sumofend = sumofend[['도착지역' ,'도착합계']]

sumofend

In [ ]:
data2 = pd.merge(data, sumofstart, on="출발지역")
data2.head()

In [ ]:
data3 = pd.merge(data2, sumofend, on="도착지역")
data3 = data3.drop_duplicates()
data3

In [ ]:
import pandas as pd

pal = pd.read_excel("/content/drive/MyDrive/학술제/location.xlsx")

pal = pal[pal.시도 =='서울특별시']
pal = pal[['시군구','읍면동','위도','경도']]
for i in pal.index:
  if pal['읍면동'][i][-1] != "동":
    pal['읍면동'][i] = pal['읍면동'][i] + '동'
pal['지역'] = pal['시군구'] + pal['읍면동']
pal = pal.drop_duplicates(['지역'])

pal1 = pal[['지역','위도','경도']]
pal1 = pal1.rename(columns={'지역':'출발지역','위도':'출발위도', '경도':'출발경도'})

pal2 = pal[['지역','위도','경도']]
pal2 = pal2.rename(columns={'지역':'도착지역','위도':'도착위도', '경도':'도착경도'})

pal2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

report = pd.read_excel('/content/drive/MyDrive/학술제/Report.xls')

In [ ]:
report = report [['자치구','동','합계']]
report.drop(report.loc[report['동']=='동'].index, inplace=True)
report.drop(report.loc[report['동']=='합계'].index, inplace=True)
report.drop(report.loc[report['동']=='소계'].index, inplace=True)
report.sort_values(by='합계', ascending=False)

In [ ]:
pal = pd.read_excel("/content/drive/MyDrive/학술제/location.xlsx")

In [ ]:
pal = pal[pal.시도 =='서울특별시']
pal = pal[['코드','시군구','읍면동','위도','경도']]
pal = pal.rename(columns={'읍면동':'동'})

In [ ]:
data = pd.merge(pal, report, on="동")
data = data.drop_duplicates()

In [ ]:
import folium
map = folium.Map(location=[37.51883692578755,126.98948],zoom_start=12)
for i in data.index:
  folium.CircleMarker([data['위도'][i],data['경도'][i]],radius=data['합계'][i]/100, popup=data['동'][i],color="#3443CC",fill_color="#3443CC").add_to(map)

In [ ]:
map

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pip install geopandas

In [ ]:
import os 
from geopandas import GeoDataFrame
import pickle
from shapely.geometry import Point
cur_path = os.getcwd() # 경로 설정

## 저상버스

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/학술제/서울특별시_저상버스 도입 노선 및 노선별 보유율_12_31_2020.csv', encoding='cp949')

In [ ]:
df.sort_values(by='보유율', ascending=True).head(10) # 저상 버스가 부족한 노선번호 상위 10개

## 지하철역

In [ ]:
subway_crd = pd.read_csv('/content/drive/MyDrive/학술제/지하철역_좌표.csv', encoding='cp949')
subway_line= pd.read_csv('/content/drive/MyDrive/학술제/서울교통공사 노선별 지하철역 정보 (2).csv', encoding='cp949')

In [ ]:
subway_crd

In [ ]:
a=subway_line[subway_line['호선'].str.contains('서해선')].index
b=subway_line[subway_line['호선'].str.contains('김포도시철도')].index
c=subway_line[subway_line['호선'].str.contains('인천선')].index
d=subway_line[subway_line['호선'].str.contains('인천2호선')].index
subway_line.drop(a,inplace=True)
subway_line.drop(b,inplace=True)
subway_line.drop(c,inplace=True)
subway_line.drop(d,inplace=True)

In [ ]:
subway_line 

In [ ]:
line_ls = []
for i in range(len(subway_crd)): 
    cur_subway = subway_crd.iloc[i]['역이름']
    matched_line = subway_line.loc[subway_line['전철역명'] == cur_subway]
    
    if len(matched_line) > 0 : 
        line_ls.append(list(matched_line['호선']))
        
    else: 
        line_ls.append(None)

subway_crd['호선'] = line_ls # 호선 정보 부여
subway_crd.dropna(subset=['호선'], inplace=True, axis=0) # 수도권 지하철역 좌표는 노선 정보가 안들어갔으므로 제거 

subway_crd['geometry'] = subway_crd.apply(lambda x: Point(x['x'], x['y']), axis=1) # 좌표 정보를 합쳐서 geometry 정보 부여  
subway_crd_gdf = GeoDataFrame(subway_crd, crs='EPSG:4326', geometry='geometry') # 좌표계 지정

In [ ]:
!pip install folium

In [ ]:
subway_crd

In [ ]:
lift = pd.read_csv('/content/drive/MyDrive/학술제/서울시 지하철 출입구 리프트 위치정보.csv', encoding='cp949')

In [ ]:
lift_location=lift['노드 WKT']

In [ ]:
lift.rename(columns = {'노드 WKT':'coordinate'},inplace=True)

In [ ]:
lift['x'] = lift.coordinate.str.split(' ').str[0]
lift['y'] = lift.coordinate.str.split(' ').str[1]

In [ ]:
lift['x']=lift['x'].str[6:]

In [ ]:
elevator = pd.read_csv('/content/drive/MyDrive/학술제/서울시 지하철역 엘리베이터 위치정보.csv', encoding='cp949')

In [ ]:
elevator.rename(columns = {'노드 WKT':'coordinate'},inplace=True)

In [ ]:
elevator['x'] = elevator.coordinate.str.split(' ').str[0]
elevator['y'] = elevator.coordinate.str.split(' ').str[1]

In [ ]:
elevator['x'] = elevator['x'].str.replace(pat=r'(', repl=r'', regex=True)
elevator['x'] = elevator['x'].str.replace(pat=r'POINT', repl=r'', regex=True)
elevator['y'] = elevator['y'].str.replace(pat=r')', repl=r'', regex=True)
lift['x'] = lift['x'].str.replace(pat=r'(', repl=r'', regex=True)
lift['y'] = lift['y'].str.replace(pat=r')', repl=r'', regex=True)

In [ ]:
elevator['x'] = elevator['x'].str.replace(pat=r'POINT', repl=r'', regex=True)
lift['x'] = lift['x'].str.replace(pat=r'POINT', repl=r'', regex=True)

In [ ]:
lift['y'] = lift['y'].astype(float)
lift['x'] = lift['x'].astype(float)

In [ ]:
elevator['y'] = elevator['y'].astype(float)
elevator['x'] = elevator['x'].astype(float)

In [ ]:
import folium
map = folium.Map(location=[37.51883692578755,126.98948],zoom_start=12)
for i in subway_crd.index: # 지하철
  folium.CircleMarker([subway_crd_gdf['y'][i],subway_crd_gdf['x'][i]], popup=subway_crd_gdf['역이름'][i],radius=2,color="#000080",fill_color="#3443CC").add_to(map)
for i in elevator.index: # 엘리베이터
  folium.CircleMarker([elevator['y'][i],elevator['x'][i]],popup=elevator['읍면동명'][i],radius=2,color="#FF0000",fill_color="#3443CC").add_to(map)
for i in lift.index: # 리프트
  folium.CircleMarker([lift['y'][i],lift['x'][i]], popup=lift['읍면동명'][i],radius=2,color="#9400D3",fill_color="#3443CC").add_to(map)

In [ ]:
map1 = folium.Map(location=[37.51883692578755,126.98948],zoom_start=12)
for i in lift.index: # 리프트
  folium.CircleMarker([lift['y'][i],lift['x'][i]], popup=lift['읍면동명'][i],radius=2,color="#9400D3",fill_color="#3443CC").add_to(map1)

In [ ]:
map1

In [ ]:
# red: subway
# purple:lift
# blue: elevator

In [ ]:
elevator.info()

In [ ]:
lift.info()

In [ ]:
map

In [ ]:
elevator.head()

In [ ]:
subway_crd.rename(columns = {'역이름' : '지하철역명'}, inplace = True)

In [ ]:
subway_crd.head()

In [ ]:
elevator['지하철역명']

In [ ]:
lift['지하철역명']

In [ ]:
subele=pd.merge(subway_crd, elevator, on = '지하철역명')
subele

In [ ]:
import folium
map = folium.Map(location=[37.51883692578755,126.98948],zoom_start=12)
for i in subway_crd.index: # 지하철
  folium.CircleMarker([subway_crd_gdf['y'][i],subway_crd_gdf['x'][i]], popup=subway_crd_gdf['지하철역명'][i],radius=2,color="#000080",fill_color="#3443CC").add_to(map)
for i in subele.index: # 지하철엘베병합
  folium.CircleMarker([subele['y_x'][i],subele['x_x'][i]], popup=subele['지하철역명'][i],radius=2,color="#FF0000",fill_color="#3443CC").add_to(map)
for i in mlift.index: # 지하철리프트병합
  folium.CircleMarker([mlift['y_x'][i],mlift['x_x'][i]], popup=mlift['지하철역명'][i],radius=2,color="#9400D3",fill_color="#3443CC").add_to(map)

In [ ]:
map

In [ ]:
# 파란색 엘리베이터 없는 역

In [ ]:
lift.head()

In [ ]:
lift.info()

In [ ]:
mlift=pd.merge(subele, lift, on = '지하철역명')

In [ ]:
mlift

In [ ]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [ ]:
from folium.plugins import MarkerCluster

In [ ]:
m5 = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m5)

for i in subway_crd.index: # 지하철
  folium.CircleMarker([subway_crd_gdf['y'][i],subway_crd_gdf['x'][i]], popup=subway_crd_gdf['지하철역명'][i],radius=2,color="#000080",fill_color="#3443CC").add_to(m5)
for i in subele.index: # 지하철엘베병합
  folium.CircleMarker([subele['y_x'][i],subele['x_x'][i]], popup=subele['지하철역명'][i],radius=2,color="#FF0000",fill_color="#3443CC").add_to(m5)
m5

In [ ]:
m5 = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m5)

for i in subway_crd.index: # 지하철
  folium.CircleMarker([subway_crd_gdf['y'][i],subway_crd_gdf['x'][i]], popup=subway_crd_gdf['지하철역명'][i],radius=2,color="#000080",fill_color="#3443CC").add_to(m5)
for i in subele.index: # 지하철엘베병합
  folium.CircleMarker([subele['y_x'][i],subele['x_x'][i]], popup=subele['지하철역명'][i],radius=2,color="#FF0000",fill_color="#3443CC").add_to(m5)
for i in mlift.index: # 지하철리프트병합
  folium.CircleMarker([mlift['y_x'][i],mlift['x_x'][i]], popup=mlift['지하철역명'][i],radius=2,color="#9400D3",fill_color="#3443CC").add_to(m5)
m5

In [ ]:
data

In [ ]:
m6 = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m6)

for i in subway_crd.index: # 지하철
  folium.CircleMarker([subway_crd_gdf['y'][i],subway_crd_gdf['x'][i]], popup=subway_crd_gdf['지하철역명'][i],radius=2,color="#000080",fill_color="#3443CC").add_to(m6)
for i in elevator.index: # 엘리베이터
  folium.CircleMarker([elevator['y'][i],elevator['x'][i]],popup=elevator['읍면동명'][i],radius=2,color="#FF0000",fill_color="#3443CC").add_to(m6)
for i in lift.index: # 리프트
  folium.CircleMarker([lift['y'][i],lift['x'][i]], popup=lift['읍면동명'][i],radius=2,color="#9400D3",fill_color="#3443CC").add_to(m6)
for i in data.index:
  folium.CircleMarker([data['위도'][i],data['경도'][i]],radius=data['합계'][i]/100, popup=data['동'][i],color="#FFFF00",fill_color="# FFFF00").add_to(m6)
m6

In [ ]:
elevator

In [ ]:
import requests
import json
import folium

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [ ]:
folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(map)

In [ ]:
for i in data3.index:
  folium.CircleMarker([data3['출발위도'][i],data3['출발경도'][i]],radius=data3['출발합계'][i]/50,color="#3443CC",fill_color="#3443CC").add_to(map)
  folium.CircleMarker([data3['도착위도'][i],data3['도착경도'][i]],radius=data3['도착합계'][i]/50,color="red",fill_color="red").add_to(map)

map